![variamos_logo](./images/variamos_logo.png)

# 2. Feature Model to XCSP

`Preamble Here`

## Table of Contents

<a href='#z_logic_rep'>2.1. Propositional Logic Representation of Feature Models</a>  
<a href='#num_rep'>2.2. Numeric Representation of Feature Models</a>  

### Import Libraries

In [1]:
import xml.dom.minidom

from jinja2 import Template

### Run Notebooks

In [2]:
%run "1 - JSON Representation of Feature Models.ipynb"

### Jupyter Notebook Styles

In [3]:
#%%html

#<style>
#    table {
#        float:left
#    }
#</style>

## 2.1. Propositional Logic Representation of Feature Models <a id='z_logic_rep'></a>

`Text Here`

| Relationship | Description |
| --- | --- |
| Mandatory | $a \leftrightarrow b$ |
| Optional | $a \to b$ |
| Requires | $ a\to b$ |
| Exclusion | $\neg(a \wedge b)$ |
| Group cardinality | $???$ |  

In [4]:
prop_log_xcsp_template = """
<instance format="XCSP3" type="CSP">
    <variables>
      <array id="a" size="{{ num_var }}" note="a[i] is the representation of each feature"> 
          0..{{ num_var - 1 }}
      </array>
    </variables>
  
    <constraints>
        <group note="Mandatory Constraints">
            <intension> iff(%0, %1) </intension>
            {%- for feature in features -%}
            {%- for constraint in feature.get("constraints", [])-%}
            {%- if constraint.get("feature_type") == "mandatory" -%}
            {% for destination in constraint.get("destination") %}
            <args> a[{{feature.get("id")}}] a[{{destination}}] </args>
            {% endfor %}
            {%- endif -%}
            {%- endfor -%}
            {% endfor %}
        </group>
    
        <group note="Optional Constraints">
            <intension> imp(%0, %1) </intension>
            {%- for feature in features -%}
            {%- for constraint in feature.get("constraints", [])-%}
            {%- if constraint.get("feature_type") == "optional" -%}
            {% for destination in constraint.get("destination") %}
            <args> a[{{feature.get("id")}}] a[{{destination}}] </args>
            {% endfor %}
            {%- endif -%}
            {%- endfor -%}
            {% endfor %}
        </group>
        
        <group note="Requires Constraints">
            <intension> imp(%0, %1) </intension>          
            {%- for feature in features -%}
            {%- for constraint in feature.get("constraints", [])-%}
            {%- if constraint.get("feature_type") == "requires" -%}
            {% for destination in constraint.get("destination") %}
            <args> a[{{feature.get("id")}}] a[{{destination}}] </args>
            {% endfor %}
            {%- endif -%}
            {%- endfor -%}
            {% endfor %}            
        </group>
        
        <group note="Exclusion Constraints">
            <intension> not(and(%0, %1)) </intension>
            {%- for feature in features -%}
            {%- for constraint in feature.get("constraints", [])-%}
            {%- if constraint.get("feature_type") == "excludes" -%}
            {% for destination in constraint.get("destination") %}
            <args> a[{{feature.get("id")}}] a[{{destination}}] </args>
            {% endfor %}
            {%- endif -%}
            {%- endfor -%}
            {% endfor %}  
        </group>
   </constraints>
</instance>
"""


def create_prop_log_XCSP(feature_model):
    features = feature_model.get('features')
    
    return Template(prop_log_xcsp_template).render(
        num_var=len(features),
        features=features,
    )

print(create_prop_log_XCSP(feature_model))


<instance format="XCSP3" type="CSP">
    <variables>
      <array id="a" size="11" note="a[i] is the representation of each feature"> 
          0..10
      </array>
    </variables>
  
    <constraints>
        <group note="Mandatory Constraints">
            <intension> iff(%0, %1) </intension>
            <args> a[1] a[4] </args>
            
            <args> a[3] a[7] </args>
            
            <args> a[3] a[8] </args>
            
        </group>
    
        <group note="Optional Constraints">
            <intension> imp(%0, %1) </intension>
            <args> a[1] a[2] </args>
            
            <args> a[1] a[3] </args>
            
            <args> a[2] a[5] </args>
            
            <args> a[2] a[6] </args>
            
        </group>
        
        <group note="Requires Constraints">
            <intension> imp(%0, %1) </intension>
            <args> a[2] a[3] </args>
                        
        </group>
        
        <group note="Exclusio

## 2.2. Numeric Representation of Feature Models <a id='num_rep'></a>  

`Text Here`

| Relationship | Description |
| --- | --- |
| Mandatory | $a = b$ |
| Optional | $a \geq b$  |
| Requires | $(1 - a + b) > 0$ |
| Exclusion | $a + b \leq 1$ |
| Group cardinality | $T_l * a \leq a+ b \leq T_h * b$ |  

In [5]:
prop_numeric_template = """
<instance format="XCSP3" type="CSP">
    <variables>
      <array id="a" size="{{ num_var }}" note="a[i] is the representation of each feature"> 
          0..{{ num_var - 1 }}
      </array>
    </variables>
  
    <constraints>
        <group note="Mandatory Constraints">
            <intension> eq(%0, %1) </intension>
            {%- for feature in features -%}
            {%- for constraint in feature.get("constraints", [])-%}
            {%- if constraint.get("feature_type") == "mandatory" -%}
            {% for destination in constraint.get("destination") %}
            <args> a[{{feature.get("id")}}] a[{{destination}}] </args>
            {% endfor %}
            {%- endif -%}
            {%- endfor -%}
            {% endfor %}
        </group>
    
        <group note="Optional Constraints">
            <intension> ge(%0, %1) </intension>
            {%- for feature in features -%}
            {%- for constraint in feature.get("constraints", [])-%}
            {%- if constraint.get("feature_type") == "optional" -%}
            {% for destination in constraint.get("destination") %}
            <args> a[{{feature.get("id")}}] a[{{destination}}] </args>
            {% endfor %}
            {%- endif -%}
            {%- endfor -%}
            {% endfor %}
        </group>
        
        <group note="Requires Constraints">
            <intension> gt(add(-1, %0, %1), 0) </intension>          
            {%- for feature in features -%}
            {%- for constraint in feature.get("constraints", [])-%}
            {%- if constraint.get("feature_type") == "requires" -%}
            {% for destination in constraint.get("destination") %}
            <args> a[{{feature.get("id")}}] a[{{destination}}] </args>
            {% endfor %}
            {%- endif -%}
            {%- endfor -%}
            {% endfor %}            
        </group>
        
        <group note="Exclusion Constraints">
            <intension> le(sum(%0, %1)), 1) </intension>
            {%- for feature in features -%}
            {%- for constraint in feature.get("constraints", [])-%}
            {%- if constraint.get("feature_type") == "excludes" -%}
            {% for destination in constraint.get("destination") %}
            <args> a[{{feature.get("id")}}] a[{{destination}}] </args>
            {% endfor %}
            {%- endif -%}
            {%- endfor -%}
            {% endfor %}  
        </group>
   </constraints>
</instance>
"""

def create_numeric_XCSP(feature_model):
    features = feature_model.get('features')
    
    return Template(prop_numeric_template).render(
        num_var=len(features),
        features=features,
    )

print(create_numeric_XCSP(feature_model))


<instance format="XCSP3" type="CSP">
    <variables>
      <array id="a" size="11" note="a[i] is the representation of each feature"> 
          0..10
      </array>
    </variables>
  
    <constraints>
        <group note="Mandatory Constraints">
            <intension> eq(%0, %1) </intension>
            <args> a[1] a[4] </args>
            
            <args> a[3] a[7] </args>
            
            <args> a[3] a[8] </args>
            
        </group>
    
        <group note="Optional Constraints">
            <intension> ge(%0, %1) </intension>
            <args> a[1] a[2] </args>
            
            <args> a[1] a[3] </args>
            
            <args> a[2] a[5] </args>
            
            <args> a[2] a[6] </args>
            
        </group>
        
        <group note="Requires Constraints">
            <intension> gt(add(-1, %0, %1), 0) </intension>
            <args> a[2] a[3] </args>
                        
        </group>
        
        <group note=